In [127]:
import pyscf
import pyscf.tools

from orbitalpartitioning import *

In [4]:
molecule = """
Fe  5.48 1.15 -8.03
S   4.05 -0.61 -8.75
S   7.49 0.42 -9.04
Fe  6.04 -1.22 -9.63
S   5.47 1.25 -5.58
S   4.63 3.28 -8.77
S   5.75 -1.50 -12.05
S   6.86 -3.41 -8.86
C   5.51 4.45 -7.51
H   6.49 4.83 -7.92
H   4.87 5.33 -7.25
H   5.72 3.84 -6.59
C   3.60 1.70 -5.54
H   3.01 0.80 -5.82
H   3.28 2.06 -4.52
H   3.42 2.48 -6.31
C   5.21 -4.22 -9.46
H   5.10 -4.01 -10.55
H   5.21 -5.32 -9.26
H   4.37 -3.72 -8.93
C   7.63 -1.85 -12.24
H   7.90 -2.06 -13.31
H   8.20 -0.96 -11.86
H   7.89 -2.72 -11.59
"""

basis = "def2-svp"
pymol = pyscf.gto.Mole(
        atom    =   molecule,
        symmetry=   True,
        spin    =   10, # number of unpaired electrons
        charge  =   -2,
        basis   =   basis)


pymol.build()
print("symmetry: ",pymol.topgroup)
# mf = pyscf.scf.UHF(pymol).x2c()
mf = pyscf.scf.ROHF(pymol).newton()
# mf = pyscf.scf.ROHF(pymol)
mf.verbose = 4
mf.conv_tol = 1e-8
mf.conv_tol_grad = 1e-5
mf.chkfile = "scf.fchk"
mf.init_guess = "sad"
mf.run(max_cycle=100)

print(" Hartree-Fock Energy: %12.8f" % mf.e_tot)
# mf.analyze()
# Get data
F = mf.get_fock()
C = mf.mo_coeff
S = mf.get_ovlp()

# Just use alpha orbitals
Cdocc = mf.mo_coeff[:,mf.mo_occ==2]
Csing = mf.mo_coeff[:,mf.mo_occ==1]
Cvirt = mf.mo_coeff[:,mf.mo_occ==0]
ndocc = Cdocc.shape[1]
nsing = Csing.shape[1]
nvirt = Cvirt.shape[1]



symmetry:  C1


******** <class 'pyscf.scf.rohf.ROHF'> Newton solver flags ********
SCF tol = 1e-08
conv_tol_grad = 1e-05
max. SCF cycles = 100
direct_scf = True
direct_scf_tol = 1e-13
chkfile to save SCF result = scf.fchk
max_cycle_inner = 12
max_stepsize = 0.05
ah_start_tol = 1e+09
ah_level_shift = 0
ah_conv_tol = 1e-12
ah_lindep = 1e-14
ah_start_cycle = 1
ah_max_cycle = 40
ah_grad_trust_region = 2.5
kf_interval = 4
kf_trust_region = 5
canonicalization = True
max_memory 4000 MB (current use 0 MB)
  HOMO = 0.199929094185569  LUMO = 0.205421971066361
Initial guess E= -5039.93568418244  |g|= 6.96962
macro= 0  E= -5052.88911939278  delta_E= -12.9534  |g|= 6.63009  3 KF 15 JK
macro= 1  E= -5062.35298948856  delta_E= -9.46387  |g|= 2.26549  3 KF 15 JK
macro= 2  E= -5064.43902264666  delta_E= -2.08603  |g|= 0.928976  3 KF 15 JK
macro= 3  E= -5066.30222790804  delta_E= -1.86321  |g|= 0.431233  3 KF 16 JK
macro= 4  E= -5067.13022699294  delta_E= -0.827999  |g|= 0.250849  3 KF 17 JK
macro= 5  

In [128]:
pyscf.tools.molden.from_mo(mf.mol, "Csing.molden", Csing)

# Define Fragments by AOs

In [138]:
# Find AO's corresponding to atoms
full = []
frag1 = []
frag2 = []
frag3 = []
frag4 = []
for ao_idx,ao in enumerate(mf.mol.ao_labels(fmt=False)):
    if ao[0] == 0:
        if ao[2] in ("3d","4s","4d"):
            frag1.append(ao_idx)
            full.append(ao_idx)
    elif ao[0] == 1:
        if ao[2] in ("3p","4p"):
            frag2.append(ao_idx)
            full.append(ao_idx)
    elif ao[0] == 2:
        if ao[2] in ("3p","4p"):
            frag3.append(ao_idx)
            full.append(ao_idx)
    elif ao[0] == 3:
        if ao[2] in ("3d","4s","4d"):
            frag4.append(ao_idx)
            full.append(ao_idx)



frags = [frag1, frag2, frag3, frag4]
print(frags)


[[3, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23], [38, 39, 40, 41, 42, 43], [56, 57, 58, 59, 60, 61], [70, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90]]


# Define Projectors
We can choose to project onto the non-orthogonal AOs, or onto the symmetrically orthogonalized AOs.

In [164]:
# Define projectors
nbas = Cdocc.shape[0]
X = scipy.linalg.sqrtm(S)
I = np.eye(nbas) 
Xinv = np.linalg.inv(X)

mat = I
Pfull = mat[:,full]  # non-orthogonal
Pf = []
for f in frags:
    Pf.append(mat[:,f])


# Get data
# F = mf.get_fock()
C = mf.mo_coeff
S = mf.get_ovlp()


# Project MOs onto all fragments
For each orbital block (Docc, Sing, Virt), project each subspace onto the full list of fragment AOs. This will determine our full active space.

In [165]:
(Oact, Sact, Vact), (Cenv, Cerr, _) = svd_subspace_partitioning((X@Cdocc, X@Csing, X@Cvirt), Pfull, I)

Oact = Xinv @ Oact
Sact = Xinv @ Sact
Vact = Xinv @ Vact

Cenv = Xinv @ Cenv 
Cerr = Xinv @ Cerr 

assert(Cerr.shape[1] == 0)
Cact = np.hstack((Oact, Sact, Vact))
pyscf.tools.molden.from_mo(mf.mol, "Cact.molden", Cact)
pyscf.tools.molden.from_mo(mf.mol, "Pfull.molden", Pfull)
print(" Should be 1: ", np.linalg.det(Cact.T @ S @ Cact))

 Partition  286 orbitals into a total of   34 orbitals
            Index   Sing. Val. Space       
                0   0.99953759            2*
                1   0.99935961            2*
                2   0.99901629            2*
                3   0.99871304            2*
                4   0.99821627            1*
                5   0.99791346            1*
                6   0.99787006            2*
                7   0.99681627            0*
                8   0.99608053            2*
                9   0.99466998            1*
               10   0.99335664            1*
               11   0.98964743            2*
               12   0.98935646            2*
               13   0.98746637            2*
               14   0.98730848            2*
               15   0.98720645            2*
               16   0.98696488            2*
               17   0.98670294            2*
               18   0.98659467            2*
               19   0.98563762            2*
 

# Split active space into fragments

In [156]:
# from pyscf import lo
# Cloc = pyscf.lo.PM(pymol).kernel(Cact, verbose=4);
# pyscf.tools.molden.from_mo(mf.mol, "Cloc.molden", Cloc)

# Cfrags = []
# Cfrags.append(Cloc[:,[i-1 for i in (1,20,22,24,25,27)]])
# Cfrags.append(Cloc[:,[i-1 for i in (19,21,23,26,28,29)]])

# for i in range(len(Cfrags)):
#     pyscf.tools.molden.from_mo(mf.mol, "Cfrag%i.molden"%i, Cfrags[i])

Set conv_tol_grad to 0.000316228
macro= 1  f(x)= 19.984704909946  delta_f= 19.9847  |g|= 1.77484  4 KF 20 Hx
macro= 2  f(x)= 23.075751573507  delta_f= 3.09105  |g|= 1.77903  3 KF 20 Hx
macro= 3  f(x)= 25.661137888061  delta_f= 2.58539  |g|= 1.62853  3 KF 20 Hx
macro= 4  f(x)= 26.594462346022  delta_f= 0.933324  |g|= 0.350775  4 KF 22 Hx
macro= 5  f(x)= 26.636442656799  delta_f= 0.0419803  |g|= 0.104438  4 KF 24 Hx
macro= 6  f(x)= 26.653910195503  delta_f= 0.0174675  |g|= 0.0438676  4 KF 26 Hx
macro= 7  f(x)= 26.658828351433  delta_f= 0.00491816  |g|= 0.0200948  4 KF 27 Hx
macro= 8  f(x)= 26.660899841946  delta_f= 0.00207149  |g|= 0.00612697  5 KF 30 Hx
macro= 9  f(x)= 26.661099226046  delta_f= 0.000199384  |g|= 0.000227938  3 KF 13 Hx
macro= 10  f(x)= 26.661099409687  delta_f= 1.83641e-07  |g|= 8.70522e-05  1 KF 2 Hx
macro X = 10  f(x)= 26.661099409687  |g|= 8.70522e-05  20 intor 35 KF 204 Hx


In [169]:
# Project active orbitals onto fragments
init_fspace = []
clusters = []
Cfrags = []
orb_index = 1


for fi,f in enumerate(frags):
    print()
    print(" Fragment: ", f)
    (Of, Sf, Vf), (_, _, _) = svd_subspace_partitioning((X@Oact, X@Sact, X@Vact), Pf[fi], I)

    Of = Xinv @ Of
    Sf = Xinv @ Sf
    Vf = Xinv @ Vf

    Cfrags.append(np.hstack((Of, Sf, Vf)))
    ndocc_f = Of.shape[1]
    init_fspace.append((ndocc_f+Sf.shape[1], ndocc_f))
    nmof = Of.shape[1] + Sf.shape[1] + Vf.shape[1]
    clusters.append(list(range(orb_index, orb_index+nmof)))
    orb_index += nmof


# Orthogonalize Fragment orbitals
Cfrags = sym_ortho(Cfrags, S)

# Pseudo canonicalize fragments
# Cfrags = orbitalpartitioning.canonicalize(Cfrags, F)


Cact = np.hstack(Cfrags)

print("nick: ", np.linalg.svd(Cact.T @ S @ Cact)[1])
# Write Molden files for visualization
pyscf.tools.molden.from_mo(mf.mol, "Pfull.molden", Pfull)
pyscf.tools.molden.from_mo(mf.mol, "Cact.molden", Cact)
pyscf.tools.molden.from_mo(mf.mol, "Cenv.molden", Cenv)
for i in range(len(frags)):
    pyscf.tools.molden.from_mo(mf.mol, "Cfrag%i.molden"%i, Cfrags[i])

print(" init_fspace = ", init_fspace)
print(" clusters    = ", clusters)




 Fragment:  [3, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]
 Partition   34 orbitals into a total of   11 orbitals
            Index   Sing. Val. Space       
                0   0.99529292            2*
                1   0.99291351            2*
                2   0.98941711            1*
                3   0.98643331            2*
                4   0.98505390            1*
                5   0.98194781            2*
                6   0.97981571            2*
                7   0.96542983            1*
                8   0.95116397            1*
                9   0.94193296            1*
               10   0.84065917            2*
               11   0.39904636            0
               12   0.34742594            0
               13   0.19373281            0
               14   0.17219181            0
               15   0.15896162            0
               16   0.05707843            1
               17   0.02939144            1
               18   0.01863364           

# Make Integrals

In [170]:
print(Cenv.shape)
print(Cact.shape)
d1_embed = 2 * Cenv @ Cenv.T

h0 = pyscf.gto.mole.energy_nuc(mf.mol)
h  = pyscf.scf.hf.get_hcore(mf.mol)
j, k = pyscf.scf.hf.get_jk(mf.mol, d1_embed, hermi=1)

print(h.shape)
h0 += np.trace(d1_embed @ ( h + .5*j - .25*k))

h = Cact.T @ h @ Cact;
j = Cact.T @ j @ Cact;
k = Cact.T @ k @ Cact;
nact = h.shape[0]

h2 = pyscf.ao2mo.kernel(pymol, Cact, aosym="s4", compact=False)
h2.shape = (nact, nact, nact, nact)
# The use of d1_embed only really makes sense if it has zero electrons in the
# active space. Let's warn the user if that's not true

S = pymol.intor("int1e_ovlp_sph")
n_act = np.trace(S @ d1_embed @ S @ Cact @ Cact.T)
if abs(n_act) > 1e-8 == False:
    print(n_act)
    error(" I found embedded electrons in the active space?!")

h1 = h + j - .5*k;

np.save("ints_h0", h0)
np.save("ints_h1", h1)
np.save("ints_h2", h2)
np.save("mo_coeffs", Cact)
np.save("overlap_mat", S)

Pa = mf.make_rdm1()[0]
Pb = mf.make_rdm1()[1]
np.save("Pa", Cact.T @ S @ Pa @ S @ Cact)
np.save("Pb", Cact.T @ S @ Pb @ S @ Cact)

(286, 82)
(286, 34)
(286, 286)


In [137]:
import numpy as np
Ccmf = np.load("Ccmf.npy")
pyscf.tools.molden.from_mo(mf.mol, "Ccmf.molden", Ccmf)
